In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
data=pd.read_csv("/content/drive/MyDrive/Cleaned_UNSW_NB15_testing-set.csv")

In [4]:
!pip install transformers

In [5]:
!pip install torch scikit-learn pandas

In [6]:
from transformers import BertTokenizer, BertForSequenceClassification
import torch
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

In [7]:
tokenizer=BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
def tokenize_features(row):
    return tokenizer(
        f"{row['proto']}  {row['service']} {row['state']} {row['sbytes']} {row['dbytes']}",
        padding='max_length',
        truncation=True,
        max_length=128,
        return_tensors='pt'
    )
tokenized_data=data.apply(tokenize_features,axis=1)
lables=data['label'].values
train_texts,test_texts,train_labels,test_labels=train_test_split(tokenized_data,lables,test_size=.2,random_state=42)
model=BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [9]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [10]:
# Combine relevant columns into a single string for each row
text_data = data.apply(
    lambda row: f"{row['proto']} {row['service']} {row['state']} {row['sbytes']} {row['dbytes']}",
    axis=1
).tolist()  # Convert to list of strings

# Tokenize the combined text data
tokenized = tokenizer(
    text_data,
    padding=True,
    truncation=True,
    max_length=128,
    return_tensors="pt"
)

# Extract tokenized components
input_ids = tokenized['input_ids']
attention_mask = tokenized['attention_mask']
labels = torch.tensor(data['label'].values)  # Ensure labels are in tensor format

from sklearn.model_selection import train_test_split

train_input_ids, test_input_ids, train_attention_mask, test_attention_mask, train_labels, test_labels = train_test_split(
    input_ids, attention_mask, labels, test_size=0.2, random_state=42
)

from torch.utils.data import TensorDataset

train_dataset = TensorDataset(train_input_ids, train_attention_mask, train_labels)
test_dataset = TensorDataset(test_input_ids, test_attention_mask, test_labels)

from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=16, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=16)

from transformers import BertForSequenceClassification

# Load BERT model with classification head
model = BertForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)


Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [11]:
import torch
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [12]:
from transformers import AdamW
# Define optimizer
optimizer = AdamW(model.parameters(), lr=5e-5)

# Define loss function (automatically handled during training in Hugging Face models)
# /usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of Adamw is deprecated and
# warnings.warn(
from torch.nn import functional as F
# Training parameters
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}")
    model.train()
    total_loss = 0
    for batch in train_dataloader:
        # Move batch to GPU
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        # Forward pass
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss
        logits = outputs.logits
        # Backward pass and optimization
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Total Loss: {total_loss}")


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:591: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1
Total Loss: 965.2953237768379
Epoch 2
Total Loss: 776.3474772493355
Epoch 3
Total Loss: 736.3186203217483


In [13]:
from sklearn.metrics import classification_report
# Put model in evaluation mode
model.eval()
# Store predictions and true labels
predictions, true_labels = [], []
with torch.no_grad():
    for batch in test_dataloader:
        batch = tuple(t.to(device) for t in batch)
        input_ids, attention_mask, labels = batch
        #Forward pass
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        #Convert logits to class predictions
        preds = torch.argmax(F.softmax(logits, dim=1), axis=1).cpu().numpy()
        predictions.extend(preds)
        true_labels.extend(labels.cpu().numpy())

# Print classification report
print(classification_report(true_labels, predictions, target_names=["Normal", "Anomalous"]))

              precision    recall  f1-score   support

      Normal       0.87      0.97      0.91      7418
   Anomalous       0.97      0.88      0.92      9049

    accuracy                           0.92     16467
   macro avg       0.92      0.92      0.92     16467
weighted avg       0.92      0.92      0.92     16467

